# 📐 Layout Detection & Text Structure Analysis

This notebook focuses on detecting document structure:
- Text blocks and paragraphs
- Line and word-level segmentation
- Reading order detection
- Table and figure detection

## Prerequisites:
```bash
pip install opencv-python numpy scipy
pip install layoutparser[effdet]  # Optional: for advanced layout detection
```

In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)
sys.path.insert(0, os.path.join(project_root, 'backend/ai-service'))

import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import patches
from PIL import Image
from dataclasses import dataclass, field
from typing import List, Dict, Any, Tuple, Optional
from scipy import ndimage
from scipy.cluster.hierarchy import fclusterdata

%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 10]

print(f"✅ Project root: {project_root}")

---
## 1️⃣ Data Structures for Layout

In [ ]:
@dataclass
class BBox:
    """Bounding box with utility methods"""
    x: int
    y: int
    w: int
    h: int
    
    @property
    def x2(self) -> int:
        return self.x + self.w
    
    @property
    def y2(self) -> int:
        return self.y + self.h
    
    @property
    def center(self) -> Tuple[int, int]:
        return (self.x + self.w // 2, self.y + self.h // 2)
    
    @property
    def area(self) -> int:
        return self.w * self.h
    
    def iou(self, other: 'BBox') -> float:
        """Intersection over Union"""
        x1 = max(self.x, other.x)
        y1 = max(self.y, other.y)
        x2 = min(self.x2, other.x2)
        y2 = min(self.y2, other.y2)
        
        if x2 <= x1 or y2 <= y1:
            return 0.0
        
        intersection = (x2 - x1) * (y2 - y1)
        union = self.area + other.area - intersection
        return intersection / union if union > 0 else 0.0
    
    def contains(self, other: 'BBox') -> bool:
        """Check if this box contains another"""
        return (self.x <= other.x and self.y <= other.y and
                self.x2 >= other.x2 and self.y2 >= other.y2)

@dataclass
class TextWord:
    """A single word with bounding box"""
    text: str
    bbox: BBox
    confidence: float = 1.0

@dataclass
class TextLine:
    """A line of text"""
    words: List[TextWord] = field(default_factory=list)
    bbox: BBox = None
    line_index: int = 0
    
    @property
    def text(self) -> str:
        return ' '.join(w.text for w in self.words)

@dataclass
class TextBlock:
    """A block of text (paragraph/section)"""
    lines: List[TextLine] = field(default_factory=list)
    bbox: BBox = None
    block_type: str = "paragraph"  # paragraph, header, list, table, figure
    block_index: int = 0
    
    @property
    def text(self) -> str:
        return '\n'.join(line.text for line in self.lines)

@dataclass
class PageLayout:
    """Complete layout of a page"""
    blocks: List[TextBlock] = field(default_factory=list)
    width: int = 0
    height: int = 0
    reading_order: List[int] = field(default_factory=list)  # Block indices in reading order

print("✅ Data structures defined")

---
## 2️⃣ Line Detection Algorithms

In [ ]:
class LineDetector:
    """Detect text lines using morphological operations"""
    
    def __init__(
        self,
        min_line_height: int = 10,
        max_line_height: int = 200,
        min_line_width: int = 50,
        horizontal_kernel_size: int = 40
    ):
        self.min_line_height = min_line_height
        self.max_line_height = max_line_height
        self.min_line_width = min_line_width
        self.horizontal_kernel_size = horizontal_kernel_size
    
    def detect(self, image: np.ndarray) -> List[BBox]:
        """
        Detect text lines in an image
        
        Returns list of BBox for each line
        """
        # Convert to grayscale
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        # Binarize
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # Dilate horizontally to connect characters
        kernel = cv2.getStructuringElement(
            cv2.MORPH_RECT, 
            (self.horizontal_kernel_size, 1)
        )
        dilated = cv2.dilate(binary, kernel, iterations=2)
        
        # Dilate vertically slightly to merge close lines
        kernel_v = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 2))
        dilated = cv2.dilate(dilated, kernel_v, iterations=1)
        
        # Find contours
        contours, _ = cv2.findContours(
            dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        
        # Filter and convert to bboxes
        lines = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            
            # Filter by size
            if h < self.min_line_height or h > self.max_line_height:
                continue
            if w < self.min_line_width:
                continue
            
            lines.append(BBox(x, y, w, h))
        
        # Sort by y position (top to bottom)
        lines.sort(key=lambda b: b.y)
        
        return lines
    
    def visualize(
        self, 
        image: np.ndarray, 
        lines: List[BBox],
        title: str = "Detected Lines"
    ):
        """Draw detected lines on image"""
        vis = image.copy()
        
        for i, line in enumerate(lines):
            color = (0, 255, 0)  # Green
            cv2.rectangle(
                vis, 
                (line.x, line.y), 
                (line.x2, line.y2), 
                color, 2
            )
            cv2.putText(
                vis, str(i), (line.x, line.y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1
            )
        
        plt.figure(figsize=(12, 10))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.title(f"{title} ({len(lines)} lines)")
        plt.axis('off')
        plt.show()

# Initialize
line_detector = LineDetector()
print("✅ Line Detector initialized")

---
## 3️⃣ Block/Paragraph Detection

In [ ]:
class BlockDetector:
    """Group text lines into blocks/paragraphs"""
    
    def __init__(
        self,
        line_spacing_threshold: float = 2.0,  # Max line gap as ratio of avg line height
        indent_threshold: int = 50  # Pixels of indent to start new paragraph
    ):
        self.line_spacing_threshold = line_spacing_threshold
        self.indent_threshold = indent_threshold
    
    def group_lines_to_blocks(
        self, 
        lines: List[BBox],
        image_height: int
    ) -> List[TextBlock]:
        """
        Group lines into logical blocks/paragraphs
        
        Uses vertical spacing and indentation cues
        """
        if not lines:
            return []
        
        # Calculate average line height
        avg_height = np.mean([line.h for line in lines])
        max_gap = avg_height * self.line_spacing_threshold
        
        # Calculate left margin (most common x position)
        x_positions = [line.x for line in lines]
        median_x = np.median(x_positions)
        
        blocks = []
        current_lines = []
        
        for i, line in enumerate(lines):
            if not current_lines:
                current_lines.append(line)
                continue
            
            prev_line = current_lines[-1]
            gap = line.y - prev_line.y2
            
            # Check if this starts a new block
            is_new_block = False
            
            # Large gap
            if gap > max_gap:
                is_new_block = True
            
            # Significant indent (paragraph start)
            if line.x - median_x > self.indent_threshold:
                is_new_block = True
            
            if is_new_block and current_lines:
                # Close current block
                blocks.append(self._create_block(current_lines, len(blocks)))
                current_lines = []
            
            current_lines.append(line)
        
        # Don't forget the last block
        if current_lines:
            blocks.append(self._create_block(current_lines, len(blocks)))
        
        return blocks
    
    def _create_block(self, lines: List[BBox], index: int) -> TextBlock:
        """Create a TextBlock from a list of line bboxes"""
        # Calculate combined bbox
        min_x = min(l.x for l in lines)
        min_y = min(l.y for l in lines)
        max_x = max(l.x2 for l in lines)
        max_y = max(l.y2 for l in lines)
        
        # Convert line bboxes to TextLine objects
        text_lines = [
            TextLine(bbox=line, line_index=i)
            for i, line in enumerate(lines)
        ]
        
        return TextBlock(
            lines=text_lines,
            bbox=BBox(min_x, min_y, max_x - min_x, max_y - min_y),
            block_type="paragraph",
            block_index=index
        )
    
    def visualize(
        self,
        image: np.ndarray,
        blocks: List[TextBlock],
        title: str = "Detected Blocks"
    ):
        """Draw detected blocks with different colors"""
        vis = image.copy()
        
        colors = [
            (255, 0, 0),    # Red
            (0, 255, 0),    # Green
            (0, 0, 255),    # Blue
            (255, 255, 0),  # Cyan
            (255, 0, 255),  # Magenta
            (0, 255, 255),  # Yellow
        ]
        
        for block in blocks:
            color = colors[block.block_index % len(colors)]
            
            # Draw block border
            cv2.rectangle(
                vis,
                (block.bbox.x, block.bbox.y),
                (block.bbox.x2, block.bbox.y2),
                color, 3
            )
            
            # Draw lines within block
            for line in block.lines:
                cv2.rectangle(
                    vis,
                    (line.bbox.x, line.bbox.y),
                    (line.bbox.x2, line.bbox.y2),
                    color, 1
                )
            
            # Label
            cv2.putText(
                vis, f"Block {block.block_index}",
                (block.bbox.x, block.bbox.y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2
            )
        
        plt.figure(figsize=(12, 10))
        plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        plt.title(f"{title} ({len(blocks)} blocks)")
        plt.axis('off')
        plt.show()

# Initialize
block_detector = BlockDetector()
print("✅ Block Detector initialized")

---
## 4️⃣ Reading Order Detection

In [ ]:
class ReadingOrderDetector:
    """Determine natural reading order of text blocks"""
    
    def __init__(self, column_threshold: float = 0.3):
        """
        Args:
            column_threshold: Horizontal overlap threshold to consider same column
        """
        self.column_threshold = column_threshold
    
    def detect_reading_order(
        self, 
        blocks: List[TextBlock],
        page_width: int
    ) -> List[int]:
        """
        Determine reading order for blocks
        
        Returns list of block indices in reading order
        """
        if not blocks:
            return []
        
        # Detect columns
        columns = self._detect_columns(blocks, page_width)
        
        # Sort: left-to-right columns, top-to-bottom within column
        ordered_indices = []
        for column in columns:
            # Sort blocks in column by y position
            column.sort(key=lambda idx: blocks[idx].bbox.y)
            ordered_indices.extend(column)
        
        return ordered_indices
    
    def _detect_columns(
        self, 
        blocks: List[TextBlock],
        page_width: int
    ) -> List[List[int]]:
        """
        Detect columns in the page layout
        
        Returns list of lists (columns containing block indices)
        """
        if len(blocks) <= 1:
            return [[i] for i in range(len(blocks))]
        
        # Get x-center of each block
        x_centers = np.array([b.bbox.center[0] for b in blocks]).reshape(-1, 1)
        
        # Cluster by x position
        threshold = page_width * self.column_threshold
        
        # Simple clustering by sorting
        indices_sorted = np.argsort(x_centers.flatten())
        
        columns = []
        current_column = [indices_sorted[0]]
        
        for i in range(1, len(indices_sorted)):
            curr_idx = indices_sorted[i]
            prev_idx = current_column[-1]
            
            # Check if horizontally close
            if abs(x_centers[curr_idx] - x_centers[prev_idx]) < threshold:
                current_column.append(curr_idx)
            else:
                columns.append(current_column)
                current_column = [curr_idx]
        
        columns.append(current_column)
        
        # Sort columns left to right
        columns.sort(key=lambda col: min(x_centers[idx][0] for idx in col))
        
        return columns

# Initialize
reading_order_detector = ReadingOrderDetector()
print("✅ Reading Order Detector initialized")

---
## 5️⃣ Complete Layout Analysis Pipeline

In [ ]:
class LayoutAnalyzer:
    """Complete document layout analysis"""
    
    def __init__(self):
        self.line_detector = LineDetector()
        self.block_detector = BlockDetector()
        self.reading_order_detector = ReadingOrderDetector()
    
    def analyze(self, image: np.ndarray) -> PageLayout:
        """
        Perform complete layout analysis on an image
        
        Returns PageLayout with all detected elements
        """
        h, w = image.shape[:2]
        
        # Step 1: Detect lines
        line_bboxes = self.line_detector.detect(image)
        
        # Step 2: Group into blocks
        blocks = self.block_detector.group_lines_to_blocks(line_bboxes, h)
        
        # Step 3: Determine reading order
        reading_order = self.reading_order_detector.detect_reading_order(blocks, w)
        
        return PageLayout(
            blocks=blocks,
            width=w,
            height=h,
            reading_order=reading_order
        )
    
    def visualize(self, image: np.ndarray, layout: PageLayout):
        """Visualize the complete layout"""
        fig, axes = plt.subplots(1, 2, figsize=(18, 10))
        
        # Original
        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Original")
        axes[0].axis('off')
        
        # Layout visualization
        vis = image.copy()
        colors = plt.cm.tab10.colors
        
        for i, block_idx in enumerate(layout.reading_order):
            block = layout.blocks[block_idx]
            color = tuple(int(c * 255) for c in colors[i % len(colors)][:3])
            
            # Draw block
            cv2.rectangle(
                vis,
                (block.bbox.x, block.bbox.y),
                (block.bbox.x2, block.bbox.y2),
                color, 3
            )
            
            # Reading order number
            cv2.putText(
                vis, str(i + 1),
                (block.bbox.x + 5, block.bbox.y + 25),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2
            )
        
        axes[1].imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f"Layout ({len(layout.blocks)} blocks, reading order numbered)")
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()

# Initialize
layout_analyzer = LayoutAnalyzer()
print("✅ Layout Analyzer ready")

In [ ]:
# Test with a synthetic image
def create_test_document():
    """Create a test document with multiple blocks"""
    img = np.ones((1200, 800, 3), dtype=np.uint8) * 255
    
    # Block 1: Header
    cv2.putText(img, "Document Title", (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1.5, (30, 30, 30), 2)
    
    # Block 2: Paragraph 1
    for i, y in enumerate(range(120, 280, 35)):
        x_offset = 80 if i == 0 else 50  # Indent first line
        cv2.line(img, (x_offset, y), (750, y), (50, 50, 50), 2)
    
    # Block 3: Paragraph 2
    for i, y in enumerate(range(340, 500, 35)):
        x_offset = 80 if i == 0 else 50
        cv2.line(img, (x_offset, y), (750, y), (50, 50, 50), 2)
    
    # Block 4: Two-column section
    # Left column
    for y in range(560, 800, 30):
        cv2.line(img, (50, y), (350, y), (50, 50, 50), 2)
    
    # Right column
    for y in range(560, 800, 30):
        cv2.line(img, (420, y), (750, y), (50, 50, 50), 2)
    
    # Block 5: Footer
    for y in range(900, 1000, 35):
        cv2.line(img, (50, y), (750, y), (50, 50, 50), 2)
    
    return img

test_doc = create_test_document()

# Analyze layout
layout = layout_analyzer.analyze(test_doc)

print(f"📐 Layout Analysis Results:")
print(f"   Image size: {layout.width}x{layout.height}")
print(f"   Blocks detected: {len(layout.blocks)}")
print(f"   Reading order: {layout.reading_order}")

# Visualize
layout_analyzer.visualize(test_doc, layout)

---
## 6️⃣ Export Layout to JSON (for API)

In [ ]:
import json

def layout_to_json(layout: PageLayout, page_id: str) -> Dict[str, Any]:
    """Convert PageLayout to JSON-serializable dict"""
    return {
        "page_id": page_id,
        "width": layout.width,
        "height": layout.height,
        "reading_order": layout.reading_order,
        "blocks": [
            {
                "block_index": block.block_index,
                "block_type": block.block_type,
                "bbox": {
                    "x": block.bbox.x,
                    "y": block.bbox.y,
                    "w": block.bbox.w,
                    "h": block.bbox.h
                },
                "lines": [
                    {
                        "line_index": line.line_index,
                        "bbox": {
                            "x": line.bbox.x,
                            "y": line.bbox.y,
                            "w": line.bbox.w,
                            "h": line.bbox.h
                        },
                        "text": line.text
                    }
                    for line in block.lines
                ]
            }
            for block in layout.blocks
        ]
    }

# Export test layout
layout_json = layout_to_json(layout, "test_page_001")
print("📄 Layout JSON:")
print(json.dumps(layout_json, indent=2)[:1500])

---
## 🎯 Summary

This notebook demonstrated:

| Component | Purpose | Output |
|-----------|---------|--------|
| LineDetector | Find text lines | BBox list |
| BlockDetector | Group lines into paragraphs | TextBlock list |
| ReadingOrderDetector | Natural reading sequence | Ordered indices |
| LayoutAnalyzer | Complete pipeline | PageLayout object |

### Integration:
- Use `LayoutAnalyzer` after image enhancement
- Feed layout data to OCR for per-line extraction
- Store layout JSON in metadata DB